In [351]:
import logging, xgboost as xgb, numpy as np
from clipper_admin import ClipperConnection, DockerContainerManager
cl = ClipperConnection(DockerContainerManager())
cl.start_clipper()
# We will register it to deploy an xgboost model.
cl.register_application('xgboost-test', 'integers', 'default_pred', 10000000)

18-07-05:11:04:11 INFO     [docker_container_manager.py:119] Starting managed Redis instance in Docker
18-07-05:11:04:15 INFO     [clipper_admin.py:126] Clipper is running
18-07-05:11:04:15 INFO     [clipper_admin.py:201] Application xgboost-test was successfully registered


In [352]:
%%notify
def get_test_point():
    return [np.random.randint(255) for _ in range(7)]
# Create a training matrix.
dtrain = xgb.DMatrix(get_test_point(), label=[0])
# We then create parameters, watchlist, and specify the number of rounds
# This is code that we use to build our XGBoost Model, and your code may differ.
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
watchlist = [(dtrain, 'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)
def predict(xs):
    return bst.predict(xgb.DMatrix(xs))
from clipper_admin.deployers import python as python_deployer
# We specify which packages to install in the pkgs_to_install arg.
# For example, if we wanted to install xgboost and psycopg2, we would use
# pkgs_to_install = ['xgboost', 'psycopg2']
python_deployer.deploy_python_closure(cl, name='xgboost-model', version=1,
     input_type="integers", func=predict, pkgs_to_install=['xgboost'])
cl.link_model_to_app('xgboost-test', 'xgboost-model')

[0]	train-error:0
[1]	train-error:0


18-07-05:11:04:15 INFO     [deployer_utils.py:44] Saving function to /tmp/clipper/tmprz79rv33
18-07-05:11:04:15 INFO     [deployer_utils.py:54] Serialized and supplied predict function
18-07-05:11:04:15 INFO     [python.py:192] Python closure saved
18-07-05:11:04:15 INFO     [python.py:206] Using Python 3.6 base image
18-07-05:11:04:15 INFO     [clipper_admin.py:452] Building model Docker image with model data from /tmp/clipper/tmprz79rv33
18-07-05:11:04:26 INFO     [clipper_admin.py:456] {'stream': 'Step 1/3 : FROM clipper/python36-closure-container:develop'}
18-07-05:11:04:26 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-07-05:11:04:26 INFO     [clipper_admin.py:456] {'stream': ' ---> c07bf98a22ab\n'}
18-07-05:11:04:26 INFO     [clipper_admin.py:456] {'stream': 'Step 2/3 : COPY /tmp/clipper/tmprz79rv33 /model/'}
18-07-05:11:04:26 INFO     [clipper_admin.py:456] {'stream': '\n'}
18-07-05:11:04:26 INFO     [clipper_admin.py:456] {'stream': ' ---> 69d7ca1084fc\n'}
18-07-05:11:04:2

<IPython.core.display.Javascript object>

In [361]:
import requests, json
# Get Address
addr = cl.get_query_addr()
# Post Query
response = requests.post(
     "http://%s/%s/predict" % (addr, 'xgboost-test'),
     headers={"Content-type": "application/json"},
     data=json.dumps({
         'input': get_test_point()
     }))
result = response.json()
result

{'combiner_output': {'final_pred': [0.5], 'msg': 'return'}, 'query_id': 8}

In [362]:
cl.stop_all()

18-07-05:11:06:31 INFO     [clipper_admin.py:1258] Stopped all Clipper cluster and all model containers
